<a href="https://colab.research.google.com/github/nakamura196/morota/blob/main/%E3%82%B8%E3%83%A3%E3%83%91%E3%83%B3%E3%82%B5%E3%83%BC%E3%83%81%E3%81%B8%E3%81%AE%E5%95%8F%E3%81%84%E5%90%88%E3%82%8F%E3%81%9B%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper
import requests
import urllib
import json
import pprint

endpoint = "https://jpsearch.go.jp/rdf/sparql/"

sparql = SPARQLWrapper(endpoint=endpoint, returnFormat='json')

# クラスが出版物のアイテムを取得する

In [ ]:
q = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX type: <https://jpsearch.go.jp/term/type/>
SELECT distinct ?cho ?label WHERE {
    ?cho rdfs:label ?label . 
    ?cho rdf:type type:出版物 . 

    # 以下、他の書き方

    # ?cho a type:出版物 . 

    # または

    # ?cho rdfs:label ?label; a type:出版物 . 
}
LIMIT 10
"""

sparql.setQuery(q)
url = "{}?query={}&format=json&output=json&results=json".format(endpoint, urllib.parse.quote(q))
r = requests.get(url)
results = json.loads(r.text)
for obj in results["results"]["bindings"]:
  # pprint.pprint(obj)
  cho = obj["cho"]["value"]
  label = obj["label"]["value"]
  print(cho, "\t", label)

# label毎のヒストグラムを作成

In [ ]:
element_freq_map = {}
for obj in results["results"]["bindings"]:
  label = obj["label"]["value"]
  if label not in element_freq_map:
    element_freq_map[label] = 0
  element_freq_map[label] += 1

# 可視化

In [ ]:
# 日本語の文字化け対策
!pip install japanize-matplotlib

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

# 表示するデータ
Y = list(element_freq_map.values())
X = list(element_freq_map.keys())

# グラフの大きさ指定
plt.figure(figsize=(10, 10))

# 棒グラフを表示
plt.barh(X, Y)

# XとYのラベル
plt.xlabel("frequency")
plt.ylabel("element")

# タイトル表示
plt.title('Element frequency', fontsize=15)